# Public key tweaking and commitments

This notebook is about demonstrating in a simplified way how public key tweaking works (as described in [LNPBP1](https://github.com/LNP-BP/LNPBPs/blob/master/lnpbp-0001.md)) and then how it is used to embed commitments for RGB ([LNPBP2](https://github.com/LNP-BP/LNPBPs/blob/master/lnpbp-0002.md))

In [1]:
import ecc.ecc as ecc
import ecc.util as util
from os import urandom
from random import randrange
import hashlib, hmac

## Reminder: what are private and public keys

In [2]:
# a private key is a scalar (= an int) in a finite field
privkey = 1 % ecc.N
print(f"the order of the field is {ecc.N}")

# public key is a point on the elliptic curve secp256k1. 
# It is obtained by adding some point of the curve called G (like "Generator") to itself a certain amount of times (which is the private key)
print(f"Point G is equal to {ecc.G}")
pubkey = privkey * ecc.G
print(f"Pubkey for privkey {privkey} is {pubkey} or {pubkey.sec().hex()}")

the order of the field is 115792089237316195423570985008687907852837564279074904382605163141518161494337
Point G is equal to 0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
Pubkey for privkey 1 is 0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 or 0279be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798


## Let's tweak a public key

In [3]:
# we generate 2 new key pairs from some entropy, one being the key pair proper, the other the "tweak"
privkey = ecc.PrivateKey(int("f02055a7c7c7680cbb23dc76ca7fea3b851984b63d93d1d6ccf3d3a9d84a97e2", 16))
pubkey = privkey.point
print(f"Our key pair is {privkey.wif()}\n{pubkey.sec().hex()}")

tweak_priv = ecc.PrivateKey(int("664d02702812642321d33e4095c8fa7de4a1e070428af6427c784362887358bc", 16))
tweak_pub = tweak_priv.point
print(f"Our tweak is {tweak_priv.wif()}\n{tweak_pub.sec().hex()}")

Our key pair is cVdUYrWTKk1EMpW2KynyW7fFoPbPkeVRkPFemQGJ5cs1wXsdKfdv
03f679efe0d62cf1fc465edfab93e40e2341ec7be081e846271a519287bd9356e4
Our tweak is cR1ZVfrm8H5SHoQHwr25qhzFjyzcBLGbFJxFMgedR8EjTLKSqAjk
03554e6d6d97b08a2ed93d729854886a76197f9546116f69eb75fe5711fc571cd9


In [4]:
# we can then generate a new key pair out of the pubkey and the tweak
tweaked_pub = pubkey + tweak_pub
print(f"The sum of {pubkey.sec().hex()} and {tweak_pub.sec().hex()} is:\n{tweaked_pub.sec().hex()}")

The sum of 03f679efe0d62cf1fc465edfab93e40e2341ec7be081e846271a519287bd9356e4 and 03554e6d6d97b08a2ed93d729854886a76197f9546116f69eb75fe5711fc571cd9 is:
0272345d85f080cb7ec6ea9716cc16bd027bc01c72929f7bb66f191ab35786fa1e


In [5]:
# the point is that we can compute the corresponding tweaked privkey
tweaked_priv = privkey.secret + tweak_priv.secret
print(f"The tweaked private key is the sum of {privkey.secret} and {tweak_priv.secret}: \n{tweaked_priv}")

The tweaked private key is the sum of f02055a7c7c7680cbb23dc76ca7fea3b851984b63d93d1d6ccf3d3a9d84a97e2 and 664d02702812642321d33e4095c8fa7de4a1e070428af6427c784362887358bc: 
566d5817efd9cc2fdcf71ab76048e4b969bb6526801ec819496c170d60bdf46f


In [9]:
# and that the resulting private key will work for the tweaked public key
tweaked_keys = ecc.PrivateKey((privkey.secret.to_int() + tweak_priv.secret.to_int()) % ecc.N) 
print(f"tweaked privkey is {tweaked_keys.secret}")

tweaked privkey is 566d5817efd9cc2fdcf71ab76048e4baaf0c883fd0d627dd8999b87f9087af5d


In [ ]:
msg = "This is a message" # This is the message we commit to
false_msg = "This is not the message we commited to"

In [ ]:
def format_msg(msg):
    # create the lnpbp1 msg
    prefix = hashlib.sha256(b"LNPBP1").digest()
    lnpbp1_msg = prefix + hashlib.sha256(b"tag").digest() + msg.encode('utf-8')
    return lnpbp1_msg

def create_commitment(pubkey, msg):
    # implementation of LNPBP 1 (single key only)
    lnpbp1_msg = format_msg(msg)
    # HMAC s and P to get the tweaking factor f
    hmac_msg = hmac.digest(pubkey.sec(False), lnpbp1_msg, hashlib.sha256)
    f = int.from_bytes(hmac_msg, 'big')
    # assert f < n
    try:
        assert f < ecc.N
    except:
        print("ERROR: tweak overflow secp256k1 order")
    # Compute a new PrivateKey with f as secret
    return ecc.PrivateKey(f)

def verify_commitment(original_pubkey, msg, commitment):
    candidate = create_commitment(original_pubkey, msg)
    print(f"candidate is {candidate.point.sec().hex()}")
    return candidate.point + original_pubkey == commitment

tweaking_factor = create_commitment(pubkey, msg)
print(f"tweaking_factor pubkey is {tweaking_factor.point.sec(False).hex()}")
tweaked_pubkey = tweaking_factor.point + pubkey
print(f"tweaked_pubkey is {tweaked_pubkey.sec(False).hex()}")
try:
    assert verify_commitment(pubkey, msg, tweaked_pubkey) == True
except:
    print("Verification failed")
    
# We show that verification fails with another message
try:
    assert verify_commitment(pubkey, false_msg, tweaked_pubkey) == False
except:
    print("Verification with a false message should fail")

In [ ]:
# now with multiple public key
def add_pubkeys(pubkeys_list):
    pubkeys_sum = ecc.S256Point.parse(pubkeys_list[0])
    for pubkey in pubkeys_list[1:]:
        point = ecc.S256Point.parse(pubkey)
        pubkeys_sum += point
    return pubkeys_sum

def generate_pubkeys_list(n):
    pubkeys = []
    for i in range(0, n):
        privkey = ecc.PrivateKey(urandom(16))
        print(f"Pubkey {i} is {privkey.point.sec().hex()}")
        pubkeys.append(privkey.point.sec())
    return pubkeys

def rm_duplicates(pubkeys_list):
    return list(dict.fromkeys(pubkeys_list))

def pick_pubkey(pubkeys):
    index = randrange(0, len(pubkeys))
    return ecc.S256Point.parse(pubkeys[index])

def verify_commitment(original_pubkey, msg, commitment, pubkeys=None):
    if pubkeys == None:
        candidate = create_commitment(original_pubkey, msg)
    else:
        pubkeys_sum = add_pubkeys(pubkeys)
        print(f"pubkeys sum is {pubkeys_sum.sec().hex()}")
        candidate = create_commitment(pubkeys_sum, msg)
    print(f"candidate is {candidate.point.sec().hex()}")
    return candidate.point + original_pubkey == commitment

def print_pubkeys(pubkeys):
    res = ""
    for i in range(0, len(pubkeys)):
        pubkey = ecc.S256Point.parse(pubkeys[i])
        temp = "pubkey[" + str(i) + "] is " + str(pubkey.sec().hex()) + "\n"
        res += temp
    return res

pubkeys = generate_pubkeys_list(6)
pubkeys.append(pubkeys[0])
pubkeys = rm_duplicates(pubkeys)
# we need to pick the pubkey we're going to tweak from the set
Po = ecc.S256Point.parse((pubkeys[0]))
print(f"Original pubkey is {Po.sec(False).hex()}")
pubkeys_sum = add_pubkeys(pubkeys)
print(f"sum of all pubkeys is {pubkeys_sum.sec().hex()}")

In [ ]:
# now do again the commitment with multiple keys
tweaking_factor = create_commitment(pubkeys_sum, msg)
print(f"tweaking_factor pubkey is {tweaking_factor.point.sec(False).hex()}\n")
tweaked_pubkey = tweaking_factor.point + Po
print(f"tweaked_pubkey is {tweaked_pubkey.sec(False).hex()}\n")
try:
    assert verify_commitment(Po, msg, tweaked_pubkey, pubkeys) == True
except:
    print("Verification failed")